In [13]:
from unbinned_lkl import (
    plot_counts_in_energy,
    previous_limits,
    FigSetup,
    compute_ALP_absorption,
    DifferentialCounts,
    GridLikelihood
    )


%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rc('xtick', labelsize=20)   
plt.rc('ytick', labelsize=20)
plt.rcParams['axes.linewidth'] = 2.5
plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=25)

from functools import partial
from scipy.stats import chi2
import numpy as np
import astropy.units as u
from pathlib import Path
import scipy.integrate as integrate
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
from astropy.table import Table

###########__________________________##############

from gammapy.modeling import Fit
import gammapy.irf as irf
from gammapy.irf import load_cta_irfs
from gammapy.data import Observation
from gammapy.utils.random import get_random_state
 
    
    
    ####-----------------------------------###
    
    
    
# models modules
from gammapy.modeling.models import (
    Model,
    Models,
    SkyModel,
    PowerLawSpectralModel,
    PowerLawNormSpectralModel,
    PointSpatialModel,
    GaussianSpatialModel,
    TemplateSpatialModel,
    FoVBackgroundModel,
    SpectralModel,
    Parameter, 
    TemplateSpectralModel
)
# dataset modules
from gammapy.datasets import (
    MapDataset, 
    MapDatasetOnOff, 
    MapDatasetEventSampler,
    SpectrumDatasetOnOff,
    SpectrumDataset, 
    Datasets
)
from gammapy.maps import MapAxis, WcsGeom, Map, MapCoord
from gammapy.makers import MapDatasetMaker, SpectrumDatasetMaker





from gammaALPs.core import Source, ALP, ModuleList
from gammaALPs.base import environs, transfer

In [14]:
def compute_unbinned_likelihood(x, dnde,**kwargs):
    s     = dnde.signal_edisp
    marks = dnde.get_dnde(x,estimated=True)

    return -s  + np.sum(np.log(marks))

In [15]:
source     = Source(z = 0.017559, ra = '03h19m48.1s', dec = '+41d30m42s') # this is for ngc1275

pin        = np.diag((1.,1.,0.)) * 0.5
alp        = ALP(0,0) 

In [16]:
pointing = SkyCoord(150.58 ,-13.26 , frame="icrs", unit="deg")
livetime = 5 * u.hr
irfs     = load_cta_irfs("C:/Users/fredr/gammapy-tutorials/datasets/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits")
observation = Observation.create( pointing=pointing, livetime=livetime, irfs=irfs)

background.py:  97 --- WARNING: Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [18]:
#runs=2
#i=1
for i in range (1,1):
    modulelist0 = ModuleList(alp, source, pin = pin)
    modulelist0.add_propagation("ICMGaussTurb", 
              0, # position of module counted from the source. 
              nsim = 3, # number of random B-field realizations
              B0 = 10.,  # rms of B field
              n0 = 39.,  # normalization of electron density
              n2 = 4.05, # second normalization of electron density, see Churazov et al. 2003, Eq. 4
              r_abell = 500., # extension of the cluster
              r_core = 80.,   # electron density parameter, see Churazov et al. 2003, Eq. 4
              r_core2 = 280., # electron density parameter, see Churazov et al. 2003, Eq. 4
              beta = 1.2,  # electron density parameter, see Churazov et al. 2003, Eq. 4
              beta2= 0.58, # electron density parameter, see Churazov et al. 2003, Eq. 4
              eta = 0.5, # scaling of B-field with electron denstiy
              kL = 0.18, # maximum turbulence scale in kpc^-1, taken from A2199 cool-core cluster, see Vacca et al. 2012 
              kH = 9.,  # minimum turbulence scale, taken from A2199 cool-core cluster, see Vacca et al. 2012
              q = -2.80, # turbulence spectral index, taken from A2199 cool-core cluster, see Vacca et al. 2012
              seed=0 # random seed for reproducability, set to None for random seed.
             )
    modulelist0.add_propagation("EBL",1, model = 'dominguez') # EBL attenuation comes second, after beam has left cluster
    modulelist0.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS') # finally, the beam enters the Milky Way Field
    
    
    
    
    
    
    
    
    
    
    PWL_index = 2.11 # spectral index of the PWL
    E0        = 300 # GeV
    Ecut      = 560 # GeV
    amplitude = 1.54 * 1e-9* u.Unit("TeV-1 cm-2 s-1") # 10e-6 

    true_m = 30 * u.eV
    true_g = 0.5 * 1/u.GeV

    enpoints, pgg   = compute_ALP_absorption(
                    modulelist = modulelist0, # modulelist from gammaALP
                    axion_mass = true_m, # neV
                    coupling   = true_g , # 10^(-11) /GeV
                    emin       = 10,  # Gev
                    emax       = 1e5, # GeV
                    bins       = 200) # log-bins in enrgy for which computing the ALP-absorption

    enpoints, pggEBL = compute_ALP_absorption(
                    modulelist = modulelist0, # modulelist from gammaALP
                    axion_mass = 0*u.neV, # neV
                    coupling   = 0*1e-11/u.GeV , # 10^(-11) /GeV
                    emin       = 10,  # Gev
                    emax       = 1e5, # GeV
                    bins       = 200) # log-bins in enrgy for which computing the ALP-absorption

    pgg  *= np.sum(pggEBL)/np.sum(pgg)

    PWL_flux        = amplitude*(enpoints/E0)**(-PWL_index)*np.exp(-enpoints/Ecut)

    EBL_PWL_flux    = amplitude*pggEBL*(enpoints/E0)**(-PWL_index)*np.exp(-enpoints/Ecut)

    ALP_PWL_flux    = amplitude*pgg*(enpoints/E0)**(-PWL_index)*np.exp(-enpoints/Ecut)

    #################################PLOT FLUX###############################
#plt.figure(figsize=(13,8))
#plt.grid(True,which='both',linewidth=0.3)
#plt.ylabel(r'$d\Phi / dE$  [TeV-1 cm-2 s-1]',size=30)
#plt.xlabel('E [GeV]',size=30)
#plt.xlim([1e1,1e4])
#plt.ylim([1e-13,4e-6])
#plt.xscale("log")
#plt.yscale("log")
    
#plt.plot(enpoints, ALP_PWL_flux,color="black", 
#          )

    #################################PLOT FLUX####################################
    
    
    # We build the Source Model combining a Spectral and Spatial model
spectral_model      = TemplateSpectralModel(enpoints*u.Unit("GeV"),ALP_PWL_flux ,interp_kwargs={"method": "linear"}) 
spatial_model_point = PointSpatialModel(lon_0="150.58 deg", lat_0="-13.26 deg", frame="icrs")
sky_model_pntpwl    = SkyModel(spectral_model=spectral_model,spatial_model=spatial_model_point, name="point-pwl")
# we are not interested for now in simulateing the bkg so we comment this part
#bkg_model           = FoVBackgroundModel(dataset_name="my-dataset")

# finally we combine source and bkg models
models              = Models([sky_model_pntpwl]) # , bkg_model] )

energy_axis      = MapAxis.from_energy_bounds( "0.01 TeV", "100 TeV", nbin=15, per_decade=True, name="energy" )
energy_axis_true = MapAxis.from_energy_bounds( "0.01 TeV", "100 TeV", nbin=45, per_decade=True, name="energy_true")
migra_axis       = MapAxis.from_bounds(0.5, 2, nbin=150, node_type="edges", name="migra")

#  MapDatase
geom     = WcsGeom.create(frame="icrs", skydir=pointing, width=(2, 2), binsz=0.02, axes=[energy_axis])
d_empty  = MapDatasetOnOff.create( geom, energy_axis_true=energy_axis_true, migra_axis=migra_axis, name="my-dataset")
maker    = MapDatasetMaker(selection=["exposure","edisp"]) # "background" 
dataset  = maker.run(d_empty, observation)
dataset.models = models



###########DNDE###########
dnde  = DifferentialCounts(dataset)

# We Print the expected number of events in both true and estimated energy
print("Total expected counts of signal events in true energy : "+str(dnde.signal))
print("Total expected counts of signal events in estimated energy : "+str(dnde.signal_edisp))

# WE SIMULATE THE EVENTS
envents_list = dnde.simulate_energies(estimated=True) # False if edisp effects are not considered
print(len(envents_list))

# WE PLOT THE SIMULATE COUNTS
fig, ax = plt.subplots(figsize=(15,10))
#Emin, Emax    = np.min(envents_list), np.max(envents_list)
#plot_dict = dict(color='black',label="Observed dN/dE")
#ax      = plot_counts_in_energy(ax,envents_list.value, Emin.value,Emax.value,en_bins=50,**plot_dict)

#AND WE ALSO PLOT THE PREDICTED dN/dE
px,py,pa = modulelist0.run(multiprocess=2)



plot_dict = dict(color='black',linewidth=2,alpha=0.8,label="dN/dE in Estimated Energy")
ax, _ = dnde.plot(ax=ax,fig=fig,estimated=True,**plot_dict)

ax.set_ylim(bottom=1e-1)
ax.set_xlim([1e-2,1e1])
ax.legend(fontsize=20)

NameError: name 'enpoints' is not defined

In [19]:
modulelist1 = ModuleList(alp, source, pin = pin)
modulelist1.add_propagation("ICMGaussTurb", 
              0, # position of module counted from the source. 
              nsim = 10, # number of random B-field realizations
              B0 = 10.,  # rms of B field
              n0 = 39.,  # normalization of electron density
              n2 = 4.05, # second normalization of electron density, see Churazov et al. 2003, Eq. 4
              r_abell = 500., # extension of the cluster
              r_core = 80.,   # electron density parameter, see Churazov et al. 2003, Eq. 4
              r_core2 = 280., # electron density parameter, see Churazov et al. 2003, Eq. 4
              beta = 1.2,  # electron density parameter, see Churazov et al. 2003, Eq. 4
              beta2= 0.58, # electron density parameter, see Churazov et al. 2003, Eq. 4
              eta = 0.5, # scaling of B-field with electron denstiy
              kL = 0.18, # maximum turbulence scale in kpc^-1, taken from A2199 cool-core cluster, see Vacca et al. 2012 
              kH = 9.,  # minimum turbulence scale, taken from A2199 cool-core cluster, see Vacca et al. 2012
              q = -2.80, # turbulence spectral index, taken from A2199 cool-core cluster, see Vacca et al. 2012
              seed=0 # random seed for reproducability, set to None for random seed.
             )
modulelist1.add_propagation("EBL",1, model = 'dominguez') # EBL attenuation comes second, after beam has left cluster
modulelist1.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS') # finally, the beam enters the Milky Way Field
    

environs.py: 431 --- INFO: Using inputted chi
environs.py:1039 --- INFO: Using inputted chi


In [20]:
print(modulelist1.modules.keys())

['MixICMGaussTurb', 'OptDepth', 'MixGMF']


In [21]:
px1,py1,pa1 = modulelist1.run(multiprocess=2)


   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>


In [ ]:
P=px1+py1

EGeV = np.logspace(1.,3.5,250)
#print(modulelist1.EGeV.shape)
#print(P)
a=0
pold=0
#DP=pold-pnew
#plt.plot(P,DP)
for p in P:
    pnew=modulelist1.EGeV
    DP=pold-pnew
    pold=modulelist1.EGeV
#DP1=
    #plt.subplot()
    plt.semilogx(modulelist1.EGeV,p)
    #print(DP)
    #a+=1

In [ ]:
#dictionary of functions for different B field realizations
ndict={}
i=0
for p in P:
    ndict[i]=modulelist1.EGeV
    i+=1
len(ndict)
print(ndict)

In [ ]:
#compare individual values for every point
Ndict={}
for p in P:
    

In [ ]:
from gammaALPs.core import Source, ALP, ModuleList
from gammaALPs.base import environs, transfer
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
from ebltable.tau_from_model import OptDepth
from astropy import constants as c

%matplotlib inline

In [ ]:
m, g = 1.,1.
alp = ALP(m,g)

In [6]:
ngc1275 = Source(z = 0.017559, ra = '03h19m48.1s', dec = '+41d30m42s')
print (ngc1275.z)
print (ngc1275.ra, ngc1275.dec)
print (ngc1275.l, ngc1275.b)


0.017559
49.950416666666655 41.51166666666666
150.57567432060083 -13.26134354429635


In [7]:

EGeV = np.logspace(1.,3.5,250)

In [8]:

pin = np.diag((1.,1.,0.)) * 0.5

In [9]:
m = ModuleList(alp, ngc1275, pin = pin, EGeV = EGeV)
m.alp.m = 30.
m.alp.g = 0.5

In [10]:
m.add_propagation("ICMGaussTurb", 
                  0, # position of module counted from the source. 
                  nsim = 10, # number of random B-field realizations
                  B0 = 10.,  # rms of B field
                  n0 = 39.,  # normalization of electron density
                  n2 = 4.05, # second normalization of electron density, see Churazov et al. 2003, Eq. 4
                  r_abell = 500., # extension of the cluster
                  r_core = 80.,   # electron density parameter, see Churazov et al. 2003, Eq. 4
                  r_core2 = 280., # electron density parameter, see Churazov et al. 2003, Eq. 4
                  beta = 1.2,  # electron density parameter, see Churazov et al. 2003, Eq. 4
                  beta2= 0.58, # electron density parameter, see Churazov et al. 2003, Eq. 4
                  eta = 0.5, # scaling of B-field with electron denstiy
                  kL = 0.18, # maximum turbulence scale in kpc^-1, taken from A2199 cool-core cluster, see Vacca et al. 2012 
                  kH = 9.,  # minimum turbulence scale, taken from A2199 cool-core cluster, see Vacca et al. 2012
                  q = -2.80, # turbulence spectral index, taken from A2199 cool-core cluster, see Vacca et al. 2012
                  seed=0 # random seed for reproducability, set to None for random seed.
                 )
m.add_propagation("EBL",1, model = 'dominguez') # EBL attenuation comes second, after beam has left cluster
m.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS') # finally, the beam enters the Milky Way Field

environs.py: 431 --- INFO: Using inputted chi
environs.py:1039 --- INFO: Using inputted chi


In [11]:
px,py,pa = m.run(multiprocess=2)

   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>


In [12]:
pgg = px + py # the total photon survival probability

print (pgg.shape)
print (np.min(np.median(pgg, axis = 0)))
print (np.min(np.max(pgg, axis = 0)))
effect = dict(path_effects=[withStroke(foreground="w", linewidth=2)])

for p in pgg: # plot all realizations
    plt.semilogx(m.EGeV, p)

#plt.xlabel('Energy (GeV)')
#plt.ylabel('Photon survival probability')
#plt.legend(loc = 0, fontsize = 'medium')
#
#plt.annotate(r'$m_a = {0:.1f}\,\mathrm{{neV}}, g_{{a\gamma}} = {1:.1f} \times 10^{{-11}}\,\mathrm{{GeV}}^{{-1}}$'.format(m.alp.m,m.alp.g),
#             xy = (0.95,0.1), size = 'x-large', xycoords = 'axes fraction', ha = 'right',**effect)
#
#plt.gca().set_xscale('log')
#plt.gca().set_yscale('log')
#plt.subplots_adjust(left = 0.2)
#plt.savefig("pgg.png", dpi = 150)


(10, 250)
0.4926360378536917
0.6594845729548637


NameError: name 'withStroke' is not defined